Instalando pacotes

In [ ]:
!pip uninstall -y numpy
!pip install torch==2.0.1 torchvision==0.15.2 numpy==1.24.4 --force-reinstall
!pip install --upgrade scipy scikit-learn
!pip install --upgrade transformers

In [ ]:
# 1. Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# 2. Load Dataset

In [ ]:
#Combining the datasets into one dataframe
import kagglehub
path = kagglehub.dataset_download("emineyetm/fake-news-detection-datasets")
print("Path to dataset files:", path)

true_path = None
fake_path = None

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        full_path = os.path.join(dirname, filename)
        if filename.lower() == 'true.csv':
            true_path = full_path
        elif filename.lower() == 'fake.csv':
            fake_path = full_path

#Combining the datasets into one dataframe
dataframeT = pd.read_csv(true_path)
dataframeF = pd.read_csv(fake_path)
dataframeT['label'] = 1
dataframeF['label'] = 0
data = pd.concat([dataframeT, dataframeF], axis=0).sample(frac=1, random_state=42)

Path to dataset files: /kaggle/input/fake-news-detection-datasets


3. Data Cleaning

In [ ]:
#used for data cleaning
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  #removes punctuation
    text = re.sub(r'\d+', '', text)  #removes numbers
    text = text.lower()  #turns everything to lowercase
    text = ' '.join([word for word in text.split() if word not in stop_words]) #gets rid of stop words
    return text

data['text'] = data['text'].apply(clean_text) #applies changes

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


3. Preprocessing

In [ ]:
#some train test splitting, 20%
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=142857)

# 4. Bag of Words + Naive Bayes

In [ ]:
bow = CountVectorizer(max_features=5000)
X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)
bow_model = MultinomialNB()
bow_model.fit(X_train_bow, y_train)
y_pred_bow = bow_model.predict(X_test_bow)

5. TF-IDF + Logistic Regression

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
tfidf_model = LogisticRegression(max_iter=1000)
tfidf_model.fit(X_train_tfidf, y_train)
y_pred_tfidf = tfidf_model.predict(X_test_tfidf)

In [ ]:
# Carrega o modelo com vetores pré-treinados (~100MB, mais leve que Word2Vec do Google)
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 25.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


6. Word2Vec + Random Forest

In [ ]:
import spacy
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Carrega o modelo com embeddings pré-treinados (~100MB)
nlp = spacy.load("en_core_web_md")

# Função para vetorizar textos com spaCy
def vectorize_spacy(texts):
    vectors = []
    for doc in nlp.pipe(texts, disable=["ner", "parser"]):
        vectors.append(doc.vector)
    return np.array(vectors)

# Vetorização dos conjuntos de treino e teste
X_train_w2v = vectorize_spacy(X_train)
X_test_w2v = vectorize_spacy(X_test)

# Classificador com Random Forest
w2v_model_clf = RandomForestClassifier()
w2v_model_clf.fit(X_train_w2v, y_train)
y_pred_w2v = w2v_model_clf.predict(X_test_w2v)

# 7. GloVe + XGBoost

In [ ]:
# Simulando GloVe com Word2Vec para simplificar, mas pode-se usar GloVe real
X_train_glove = X_train_w2v
X_test_glove = X_test_w2v
glove_model = XGBClassifier(eval_metric='logloss')
glove_model.fit(X_train_glove, y_train)
y_pred_glove = glove_model.predict(X_test_glove)

8. BERT + Fine Tuning

In [ ]:
class FakeNewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.encodings = tokenizer(list(texts), truncation=True, padding=True, max_length=max_len)
        self.labels = list(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
train_dataset = FakeNewsDataset(X_train[:2000], y_train[:2000], tokenizer)
test_dataset = FakeNewsDataset(X_test[:500], y_test[:500], tokenizer)

training_args = TrainingArguments(output_dir="./results", per_device_train_batch_size=8, per_device_eval_batch_size=8, num_train_epochs=2, logging_dir="./logs", logging_steps=10)

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)
trainer.train()
preds = trainer.predict(test_dataset)
y_pred_bert = np.argmax(preds.predictions, axis=1)
y_true_bert = y_test[:500].values

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ffcs (ffcs-cin-ufpe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.648800
20,0.391200
30,0.260300
40,0.078200
50,0.016000
60,0.002000
70,0.003900
80,0.163700
90,0.119900
100,0.100500


# 9. Evaluation Results

In [ ]:
def evaluate(name, y_true, y_pred):
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1-score:", f1_score(y_true, y_pred))

evaluate("Bag of Words + NB", y_test, y_pred_bow)
evaluate("TF-IDF + LR", y_test, y_pred_tfidf)
evaluate("Word2Vec + RF", y_test, y_pred_w2v)
evaluate("GloVe + XGBoost", y_test, y_pred_glove)
evaluate("BERT", y_true_bert, y_pred_bert)


=== Bag of Words + NB ===
Accuracy: 0.95
Precision: 0.943759802823213
Recall: 0.9551020408163265
F1-score: 0.9493970472219092

=== TF-IDF + LR ===
Accuracy: 0.9898663697104677
Precision: 0.9886852229011088
Recall: 0.990702947845805
F1-score: 0.9896930569713445

=== Word2Vec + RF ===
Accuracy: 0.938195991091314
Precision: 0.9391660970608339
Recall: 0.9346938775510204
F1-score: 0.9369246505284692

=== GloVe + XGBoost ===
Accuracy: 0.9662583518930957
Precision: 0.9680875313426032
Recall: 0.9630385487528345
F1-score: 0.9655564396953507

=== BERT ===
Accuracy: 0.998
Precision: 1.0
Recall: 0.99581589958159
F1-score: 0.9979035639412998
